# Creating a prompt template using exclusively chatGPT  

In [37]:
import pandas as pd
from openai import OpenAI 
import json
from dotenv import load_dotenv
import os

In [26]:
# Load environment variables from .env file
load_dotenv()

# Retrieve OpenAI API key from environment variables
openai.api_key = os.getenv('OPENAI_API_KEY')


In [27]:
file_path = '/Users/carlos/Desktop/Tesis/Template/rag-templates/datasets/SWArchtitecture-classification.csv'  # Replace with the actual path to your CSV file

# Function to read CSV file
def read_csv(file_path):
    # Ensure the CSV file is read correctly, handling multi-line fields
    df = pd.read_csv(file_path, delimiter=';', quoting=1, quotechar='"', engine='python')
    df['Chapter'] = 'Specification of software safety requirements'
    return df

read_csv(file_path)

,ID,Content,Work Product,Category ID,Category Description,Process Area,Chapter
0,26262-2018:6-7.4.1,To avoid systematic faults in the software arc...,Software architectural design specification,4,Quality criteria for work products,Software Architectural Design,Specification of software safety requirements
1,26262-2018:6-7.4.10a,Safety-oriented analysis shall be carried out ...,Safety analysis report,1,Identification of work products,Software Architectural Design,Specification of software safety requirements
2,26262-2018:6-7.4.10b,"in accordance with ISO 26262-9:2018, Clause 8,...",Safety analysis report,4,Quality criteria for work products,Software Architectural Design,Specification of software safety requirements
3,26262-2018:6-7.4.11,If the implementation of software safety requi...,Dependent failures analysis report,1,Identification of work products,Software Architectural Design,Specification of software safety requirements
4,26262-2018:6-7.4.12,Depending on the results of the safety-oriente...,Software architectural design specification,1,Identification of work products,Software Architectural Design,Specification of software safety requirements
5,26262-2018:6-7.4.13,An upper estimation of required resources for ...,Software architectural design specification,2,Requirements on the content of work products,Software Architectural Design,Specification of software safety requirements
6,26262-2018:6-7.4.14a,The software architectural design shall be ver...,Software architectural design specification,9,Requirements on work product lifecycles (revie...,Software Architectural Design,Specification of software safety requirements
7,26262-2018:6-7.4.14b,and by using the software architectural desig...,Software architectural design specification,4,Quality criteria for work products,Software Architectural Design,Specification of software safety requirements
8,26262-2018:6-7.4.2,During the development of the software archite...,Software architectural design specification,4,Quality criteria for work products,Software Architectural Design,Specification of software safety requirements
9,26262-2018:6-7.4.3,"In order to avoid systematic faults, the softw...",Software architectural design specification,4,Quality criteria for work products,Software Architectural Design,Specification of software safety requirements


In [43]:
# Function to generate checklist using OpenAI's GPT-4
def generate_checklist(chapter, working_product, standards):
    standards_list = "\n".join([f"{ID}: {Content}" for ID, Content in standards.items()])
    prompt = f"""
    You are a consultant specializing in compliance and security, assisting in the release of a new product. Your task is to create comprehensive checklists that ensure adherence to various standards, specifically from the ISO and SPICE frameworks, provided in a CSV format. Each checklist should focus on fulfilling the auditing requirements for these standards.

    Chapter: {chapter}
    Working Product: {working_product}
    Standards:
    {standards_list}

    For these standards, develop a checklist that answers the following questions:

    What actions must be taken to comply with the standards' requirements?
    What documentation or evidence is needed to demonstrate compliance?
    What processes or procedures should be implemented to ensure ongoing compliance?
    How should compliance be monitored and maintained over time?

    Ensure that the checklists are detailed, actionable, and aligned with best practices for compliance and security. The goal is to facilitate a smooth and thorough auditing process in the future.

    The checklist should be returned in a JSON format and must include the IDs of the standards used for each checkpoint.
    """
    client = OpenAI()
    chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a consultant specializing in compliance and security.",
        },
        {
            "role": "user",
            "content": prompt,
        },
    ],
    model="gpt-3.5-turbo",
    n=1
    )
    print(chat_completion)
    return chat_completion



In [39]:
# Function to structure output in the required JSON format
def structure_output(df):
    grouped = df.groupby(['Chapter', 'Work Product'])
    output = []

    for (chapter, work_product), group in grouped:
        standards = dict(zip(group['ID'], group['Content']))
        checklist = generate_checklist(chapter, work_product, standards)
        
        output.append({
            "Chapter": chapter,
            "Work Product": work_product,
            "Checklist": checklist,
            "ID": list(standards.keys())
        })
    return output

In [44]:

df = read_csv(file_path)
structured_data = structure_output(df)

# Convert structured data to JSON
json_output = json.dumps(structured_data, indent=4)
print(json_output)

ChatCompletion(id='chatcmpl-9T6N3mf2KiDcxJg8QhzrQzkCr0Fhk', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n{\n    "checklist": [\n        {\n            "id": "26262-2018:6-7.4.11",\n            "standard": "ISO 26262-2018:6-7.4.11",\n            "audit_requirement": "Dependent failures analysis report",\n            "questions": [\n                {\n                    "action": "Perform analysis of dependent failures and their effects as per ISO 26262-9:2018, Clause 7",\n                    "document_needed": "Dependent failures analysis report",\n                    "process_procedure": "Establish a formal process for conducting dependent failures analysis",\n                    "compliance_monitoring": "Implement regular reviews of the dependent failures analysis report to ensure accuracy and relevance"\n                },\n                {\n                    "action": "Refer to Annex E and ISO 26262-9:2018, Annex C

TypeError: Object of type ChatCompletion is not JSON serializable